# This is the code which solves the Static Traffic Assignment Problem

We want to solve the following problem (TAP-C):
\begin{align}
\min_{f, h} &\sum_{a} \int_{0}^{f_a} c_a(s)\; \text{d}s
\\
\text{s.t.  } & \;\; f = \Delta h
\\
& \;\; 
h \geq 0
\\
& \;\; 
A h = d
\\
& \;\; 
f \leq u
\end{align}

This problem is hard to compute, even if it is a convex problem. Computing all the path of a network is NP-hard (with respect to the number of edges and links).

Nevertheless, the TAP can be solve using a Frank-Wolf algorithm:
\begin{align}
&\text{1. TO DO }
\\
&\text{2. }
\end{align}


#### Remarks
First, we tried to solve the STA with CVX.
This does not work because the values of the travel time are to big. 

Secondly, we solve the STA using links flow. 
This does not help us in our project because we need the path flow to compute the dual of the TAP-C.

Thirdly, we solve the STA using paths flow. 
The issue here is that finding every path in a network in a NP-complete problem. 
But if we can compute every path (we only need to do it one time), then it is really fast to compute the shortest path. We do not need to do a Dikjstra's algorithm to compute the shortest path, only a big matrix multiplication and a array sorting are enough. This might be more difficult on a laptop. But it might be faster on a HPC.

Finally, we solve the STA using links flow and we compute the paths during the all or nothing step of the Frank Wolf algorithm. TO BE DONE.

In [215]:
import numpy as np
import scipy.sparse

In [216]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network = I210
debug = True

## 1. We load the graph and the demand
Both graph and demand are in csv file, we load them

In [218]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

### We clean the data

In [219]:
"""
We need to demand to be an array like:
[
[o1, d1, demand from o1 to d1],
...,
[on, dn, demand from on to dn]
]

We need the graph to be an array like:
[
[id link 1, node origin link 1, node destination link 1, a1, a2, a3, a4],
...,
[id link n, node origin link n, node destination link n, a1, a2, a3, a4]
]
where the node are indexed from 0 to nb_nodes-1,
    the links are indexed from 0 to nb_links-1

One can add some checks here to be sure that demand and graph respect this format!
"""

def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix (2d array)
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    return graph, demand

graph, demand = cleaning_input(graph, demand)
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
nb_ods = int(demand.shape[0])
if debug:
    print("nb nodes = " + str(nb_nodes))
    print("nb links = " + str(nb_links))
    print("nb ods = " + str(nb_ods))

Then, we define the function which gives the travel time as a function of the flow

In [221]:
## Here Max float is fixed to not overlap the maximum float number when we compute the potential function
max_float = 1e+10
if debug:
    print(max_float)

# TO DO ADD THE CAPACITY
def travel_time(f, c = -1):
    if c == -1:
        c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    # here we need to have the same indexation for graph and f.
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else max_float for i in range(len(f))]
    return tt_tmp

## 2. We compute the all or nothing flow allocation

In [223]:
from scipy.sparse.csgraph import dijkstra

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [224]:
def update_travel_time(tt):
    for i in range(graph.shape[0]):
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]

In [225]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
update_travel_time(travel_time(np.zeros(nb_links))) # , [1,1,-1,1,1]))
# the following line make the matrix sparse, one can use G.toarray() to do the opposite
G = scipy.sparse.csr_matrix(G)
print(G)

  (0, 1)	1.38
  (0, 6)	1.15
  (1, 2)	1.38
  (1, 7)	1.14
  (2, 3)	1.68
  (2, 8)	1.13
  (3, 4)	1.26
  (3, 9)	1.05
  (4, 5)	2.25
  (4, 10)	1.04
  (5, 11)	1.09
  (6, 0)	1.15
  (6, 7)	0.48
  (6, 12)	0.36
  (7, 1)	1.14
  (7, 8)	0.48
  (7, 13)	0.39
  (8, 2)	1.13
  (8, 9)	0.58
  (8, 14)	0.38
  (9, 3)	1.05
  (9, 10)	0.43
  (9, 15)	0.48
  (10, 4)	1.04
  (10, 11)	0.77
  (10, 16)	0.5
  (11, 5)	1.09
  (11, 17)	0.47
  (11, 19)	0.22
  (12, 6)	0.36
  (12, 13)	1.38
  (13, 7)	0.39
  (13, 14)	1.38
  (14, 8)	0.38
  (14, 15)	1.68
  (15, 9)	0.48
  (15, 16)	1.25
  (16, 10)	0.5
  (16, 17)	2.22
  (17, 11)	0.47
  (18, 6)	0.29


In [226]:
class path:
    def __init__(self,links):
        self.links = links
        self.flow = 0
    def add_flow(self, flow):
        self.flow += flow
    def __eq__(self, other):
        return np.all(self.links == other.links)
    def __hash__(self):
        return hash(np.sum([hash(self.links[i]*(nb_links**i)) for i in range(len(self.links))]))
    def __str__(self):
        return str(self.flow) + " is on " + str(self.links) 

In [227]:
paths_used = {}
print(paths_used)
p = path([0, 1, 2, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)
p = path([0, 1, 1, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)
p = path([0, 0, 2, 3])
paths_used[hash(p)] = (p, 0)
print(paths_used)

{}
{210166: (<__main__.path object at 0x1102f08d0>, 0)}
{210166: (<__main__.path object at 0x1102f08d0>, 0), 208485: (<__main__.path object at 0x1102f0780>, 0)}
{210166: (<__main__.path object at 0x1102f08d0>, 0), 208485: (<__main__.path object at 0x1102f0780>, 0), 210125: (<__main__.path object at 0x1102f0a90>, 0)}


Now let's compute the all or nothing allocation

In [228]:
# computing the all or nothing flow
paths_used = {}

def all_or_nothing():
    global k
    paths_used_tmp = {}
    faon = np.zeros(nb_links)
    # using scipy to compute dijkstra
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    """faon = np.zeros(shape = nb_links)"""
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        node_tmp = d_tmp
        links = []
        while node_tmp != o_tmp:
            node_tmp_d = return_predecessors[o_tmp][node_tmp]
            # link_tmp = list(G.todok().keys()).index((node_tmp_d, node_tmp)) #I don't know how much time it takes, should I stock the list of keys in memory?
            link_tmp = graph_dict[path[i]][path[i+1]]
            links.insert(0, link_tmp)
            faon[link_tmp] += flow_tmp
            node_tmp = node_tmp_d
        
        p = path(links)
        p.add_flow(flow_tmp)
        print(p)
        if hash(p) in paths_used:
            continue
            # print(str(p) + " is already in paths_used" + str(paths_used[hash(p)]))
        else:
            print(k)
            paths_used_tmp[hash(p)] = (p, k)
            k = k+1
            
    paths_used.update(paths_used_tmp)
    return faon, paths_used_tmp

We define the line search

In [229]:
k = 0 
all_or_nothing()
print(paths_used)

TypeError: 'type' object is not subscriptable

In [230]:
# TO DO: rewrite this function
def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    links = int(np.max(graph[:, 0]) + 1)
    g = np.copy(
        graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [231]:
eps=1e-8
k = 0
paths_used = {}
f = np.zeros(nb_links)
update_travel_time(travel_time(f))

f, paths_aux = all_or_nothing()
print(f)
update_travel_time(travel_time(f))
print(paths_used)
path_matrix = np.zeros(len(paths_aux))

for val in paths_aux.values():
    print(val[1])
    path_matrix[val[1]] = val[0].flow

print(path_matrix)

for i in range(1000):
    if i % 100 == 0:
        print(i)
    faon, paths_tmp = all_or_nothing() 
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    print("s: " + str(s))
    if s < eps:
        break
    path_matrix = np.append(path_matrix, np.zeros(len(paths_used)-path_matrix.shape[0]))
    path_matrix = (1-s) * path_matrix
    for val in paths_tmp.values():
        path_matrix[val[1]] += s * val[0].flow
    f = (1. - s) * f + s * faon
    
    # also update the path flow
    ## TO DO ##
    update_travel_time(travel_time(f))
print(path_matrix)

TypeError: 'type' object is not subscriptable

In [232]:
for p in paths_used.values():
    index = p[1]
    p[0].add_flow(path_matrix[index] - p[0].flow)
    print(str(path_matrix[index]) + " is on " + str(p[0]))

print(demand)
print(G)
print(f)



[[1.8e+01 1.9e+01 2.5e+04]]
  (0, 1)	1.38
  (0, 6)	1.15
  (1, 2)	1.38
  (1, 7)	1.14
  (2, 3)	1.68
  (2, 8)	1.13
  (3, 4)	1.26
  (3, 9)	1.05
  (4, 5)	2.25
  (4, 10)	1.04
  (5, 11)	1.09
  (6, 0)	1.15
  (6, 7)	0.48
  (6, 12)	0.36
  (7, 1)	1.14
  (7, 8)	0.48
  (7, 13)	0.39
  (8, 2)	1.13
  (8, 9)	0.58
  (8, 14)	0.38
  (9, 3)	1.05
  (9, 10)	0.43
  (9, 15)	0.48
  (10, 4)	1.04
  (10, 11)	0.77
  (10, 16)	0.5
  (11, 5)	1.09
  (11, 17)	0.47
  (11, 19)	0.22
  (12, 6)	0.36
  (12, 13)	1.38
  (13, 7)	0.39
  (13, 14)	1.38
  (14, 8)	0.38
  (14, 15)	1.68
  (15, 9)	0.48
  (15, 16)	1.25
  (16, 10)	0.5
  (16, 17)	2.22
  (17, 11)	0.47
  (18, 6)	0.29
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [233]:
list(G.todok().keys()).index((7, 13))

30

In [234]:
print("[", end="")
for i in range(len(travel_time(f))):
    print(max_float, end=", ")
print()

[

NameError: name 'max_float' is not defined

In [235]:
print(graph)
print(paths_used)

[[0.0000e+00 0.0000e+00 1.0000e+00 1.3800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.3120e+00]
 [1.0000e+00 0.0000e+00 6.0000e+00 1.1500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7600e+00]
 [2.0000e+00 1.0000e+00 2.0000e+00 1.3800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.3120e+00]
 [3.0000e+00 1.0000e+00 7.0000e+00 1.1400e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7360e+00]
 [4.0000e+00 2.0000e+00 3.0000e+00 1.6800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 4.0320e+00]
 [5.0000e+00 2.0000e+00 8.0000e+00 1.1300e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7120e+00]
 [6.0000e+00 3.0000e+00 4.0000e+00 1.2600e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.0240e+00]
 [7.0000e+00 3.0000e+00 9.0000e+00 1.0500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.5200e+00]
 [8.0000e+00 4.0000e+00 5.0000e+00 2.2500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 5.4000e+00]
 [9.0000e+00 4.0000e+00 1.0000e+01 1.0400e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.4960e+00]
 [1.0000e+01 5.0000e+00 1.1000e+01 1.0900e+00 0.0000e+00 0.0000e+00
  

### The following is old

In [257]:
import sys
max_float = sys.float_info.max
print(max_float)

1.7976931348623157e+308


In [258]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    
# print(graph_dict)
# Here we remove graph_dict
del graph_dict

## Test to run the algorithm with path flow

### I. We load and clean the network and the demand data

In [80]:
import numpy as np
import scipy
import scipy.sparse

In [64]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network = I210
debug = False

In [65]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [66]:
def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    return graph, demand

graph, demand = cleaning_input(graph, demand)
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
nb_ods = int(demand.shape[0])
if debug:
    print("nb nodes = " + str(nb_nodes))

### 2. We compute the incidence matrix

In [67]:
from collections import defaultdict 
   
# This class represents a directed graph  
# using adjacency list representation 
class Graph: 
   
    def __init__(self,vertices): 
        #No. of vertices 
        self.V= vertices  
        # default dictionary to store graph 
        self.graph = defaultdict(list)  
   
    # function to add an edge to graph 
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
   
    def printAllPathsUtil(self, u, d, visited, path, graph_dict): 
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
        
        if u == d: 
            path_tmp = np.zeros(shape=(self.V))
            for i in range(len(path)-1):
                # link_indice = list(G.todok().keys()).index((path[i], path[i+1]))
                link_indice = graph_dict[path[i]][path[i+1]]
                path_tmp[link_indice] = 1
            if self.paths_m.shape[0]==0:
                self.paths_m = path_tmp.reshape((1, self.V))
            else:
                self.paths_m = np.append(self.paths_m, path_tmp.reshape((1, self.V)), axis=0)
        else: 
            # If current vertex is not destination 
            # Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]==False: 
                    self.printAllPathsUtil(i, d, visited, path, graph_dict) 
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop()
        visited[u]= False
   
   
    # Prints all paths from 's' to 'd' 
    def printAllPaths(self,s, d, graph_dict): 
        self.paths_m = np.array([])
        # Mark all the vertices as not visited 
        visited =[False]*(self.V)
  
        # Create an array to store paths 
        path = [] 
        
        # Call the recursive helper function to print all paths 
        self.printAllPathsUtil(s, d,visited, path, graph_dict) 
        return self.paths_m 

def return_graph_dict(graph):
    # graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
    graph_dict = {}
    for i in range(graph.shape[0]):
        try: 
            graph_dict[int(graph[i][1])]
        except:
            graph_dict[int(graph[i][1])] = {}
        graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    return graph_dict
    
# argument graph
def delta_matrix(graph, demand):
    route2od = np.array([])
    paths_matrix = np.array([])
    graph_dict = return_graph_dict(graph)
    g = Graph(graph.shape[0]) 
    for line in graph:
        g.addEdge(int(line[1]), int(line[2]))

    for i in range(demand.shape[0]):
        s = int(demand[i][0]) ; d = int(demand[i][1])
        paths_matrix_tmp = g.printAllPaths(s, d, graph_dict)
        route2od_tmp = np.ones(paths_matrix_tmp.shape[0])
        route2od_tmp = route2od_tmp*i
        if paths_matrix.shape[0]==0:
            paths_matrix = paths_matrix_tmp
        else:
            paths_matrix = np.append(paths_matrix, paths_matrix_tmp, axis=0)
        route2od = np.append(route2od, route2od_tmp)
    return paths_matrix, route2od

In [68]:
import sys
max_float = sys.float_info.max
if debug:
    print(max_float)

# TO DO ADD THE CAPACITY
def travel_time(f, c = -1):
    if c == -1:
        c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    # here we need to have the same indexation for graph and f.
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else float('inf') for i in range(len(f))]
    return tt_tmp

In [69]:
def update_travel_time(tt, G):
    for i in range(graph.shape[0]):
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]
    return G

In [70]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
G = update_travel_time(travel_time(np.zeros(nb_links)), G) # , [1,1,-1,1,1]))
# the following line make the matrix sparse, one can use G.toarray() to do the opposite
G = scipy.sparse.csr_matrix(G)

In [71]:
delta, route2od = delta_matrix(graph, demand)
if debug:
    print(delta[0])

In [72]:
tt = travel_time(np.zeros(nb_links))
delta_csr = scipy.sparse.csr_matrix(delta)
if debug:
    print((delta_csr @ tt))
    print(np.argmin(delta_csr @ tt))

### 3. We upload the flow allocation 
#### 3.1 We compute the travel time of each path. 
#### 3.2 We compute the all or nothing allocation by putting all the demand on shortest paths.
#### 3.3 We update the flow allocation with a bisection.

In [73]:
# check sparse vector
h = scipy.sparse.lil_matrix(np.zeros(delta_csr.shape[0]))
# the following is for one od
demand[0][2] = 10
# TO DO MODIFY TO TAKE INTO ACCOUNT SEVERALS OD
h[:,0] = demand[0][2]
print(h)

  (0, 0)	10.0


In [74]:
# TO DO: rewrite this function
def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    links = int(np.max(graph[:, 0]) + 1)
    g = np.copy(
        graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if debug:
            print(potential(graph, (1. - a) * f_tmp.toarray()[0] + a * f_aon.toarray()[0]))
            print(l * d, end=": ")
            print(f(l * d))
            print(r * d, end=": ")
            print(f(r * d))
            print(str(m1 * d) + " = " + str(f(m1 * d)))
            print(str(m2 * d) + " = " + str(f(m2 * d)))
            print()
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

In [75]:
t = 0
nb_iter = 1000
for i in range(nb_iter):
    f = h @ delta_csr
    tt_flow = travel_time(f.toarray()[0])
    tt_p = delta_csr @ tt_flow
    # the following is for one od
    p_aon = np.argmin(tt_p)
    if i % (nb_iter/20) == 0:
        print(i)
    if debug:
        print(f.toarray()[0])
        print(tt_flow)
        print(tt_p)
    # the following is for one od
    # TO MODIFY TO TAKE INTO ACCOUNT SEVERAL ODs
    h_aon = scipy.sparse.lil_matrix(np.zeros(delta_csr.shape[0]))
    h_aon[:, p_aon] = demand[0][2]
    f_aon = h_aon @ delta_csr
    t = potential(graph, f.toarray()[0])
    s = line_search(lambda a: potential(graph, (1. - a) * f.toarray()[0] + a * f_aon.toarray()[0]))
    h = (1.-s)*h + s*h_aon
    if debug:
        print()
        print(potential(graph,f.toarray()[0])-potential(graph, (1. - s) * f.toarray()[0] + s * f_aon.toarray()))
        print(s)
print(h)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
  (0, 0)	1.0066659776071194
  (0, 81)	0.041058222655458936
  (0, 82)	0.061641663928456714
  (0, 121)	7.756863747158167
  (0, 160)	0.024620126347929774
  (0, 161)	0.005959519000975836
  (0, 241)	4.7666508869616896e-05
  (0, 242)	1.1031430767930204


In [76]:
f = h @ delta_csr
tt_flow = travel_time(f.toarray()[0])
tt_p = delta_csr @ tt_flow
if debug:
    print(tt_p[h.toarray()[0]!=0])
    print(np.argsort(tt_p))
    print(h!=0)
    print(np.sort(tt_p))

In [79]:
if debug:
    print(np.argsort(delta_csr @ tt_flow))
    print((delta_csr @ tt_flow))
    print(h.toarray()[0])